# `swift_too` module

## Swift_QueryJob example - Querying and fetching the results of an existing job

### API version = 1.2

#### Author: Jamie A. Kennea (Penn State)

If you wish to retrieve the results of a previously job, this is the task of the QueryJob class. All you need for this class is your username, shared secret and the job number which is reported back when the job is submitted either with `submit()` or `queue()` methods. The jobnumber can be found in the property `jobnumber`.

In [1]:
from swifttools.swift_too import QueryJob, Swift_VisQuery
from time import sleep

I'll define our username and shared secret next because we'll use them twice in this notebook

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

First let's submit a simple job so we can get an example jobnumber. In this case I'm just going to do something that isn't too strenuous on the system, let's see if Sgr A* is currently visible, or when it is next visible.

In [3]:
vq = Swift_VisQuery()
vq.username = username
vq.shared_secret = shared_secret
vq.ra,vq.dec = 266.4168, -29.0078
vq.length = 1
vq.hires = True
if vq.queue():
    print(f"Queued job successfully! Jobnumber = {vq.status.jobnumber}")
else:
    print(f"Error: {vq.status.errors}")

Queued job successfully! Jobnumber = 47836


Note that the job has been submitted, but because we used the `queue()` method, it may not have completed yet. If you wait a bit, it will be, but the next step can be run straight away to demonstrate what happens if processing isn't complete.



OK let's see how our job is doing, because the purpose of the QueryJob class is to allow us to fetch a job later, or maybe ASAP. Maybe you don't want to hang around waiting for a bunch of jobs to complete, submit some, have some coffee, come back and fetch, or just want to query the result of an existing job.

In [4]:
qj = QueryJob()
qj.username = username
qj.shared_secret = shared_secret
qj.jobnumber = vq.status.jobnumber
qj.submit()

print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")


Current status of job #47836: Processing


Note that for the class `QueryJob`, as it does not take many parameters, you can also pass parameters as arguments. If you do this, then the request will be submitted on invokation, so there's no need use the `submit()` method.

In [5]:
while(not qj):
    qj = QueryJob(username,shared_secret,vq.status.jobnumber)
    print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")
    sleep(1)

Current status of job #47836: Accepted


The `submit` (or argument form) should return `True` if the job has been fetched. It'll return `False` for a number of reasons, firstly if the job is still *Queued* (waiting to be processed) or *Processing*, the status will indicate this. Also the status can be *Rejected*, in the case where you have given bad parameters, or the job no longer exists.


The above "while" will keep polling the job until it is complete. We put in the sleep between requests so as to not over spam the TOO_API server.

OK let's assume that the above now reports "Current status of job #XXX: Accepted", which indicates a successful `QueryJob` call of a completed job. Let's see what our QueryJob class data now looks like.

In [6]:
qj

Parameter,Value
username,myuser
status,Accepted
jobnumber,47836
errors,[]
warnings,[]
timestamp,2021-07-08 19:58:40
began,2021-07-08 19:58:40
completed,2021-07-08 19:58:41
fetchresult,True
result,Swift_VisQuery object


You can see the data inside the class here presented in table form. The the more useful value here is `result`, which contains the result of the job. In the Python api, this will be a `Swift_VisQuery` object. You can display the result of the job like this:

In [7]:
qj.result

Begin,End,Length
2021-07-08 20:03:00,2021-07-08 20:33:00,0:30:00
2021-07-08 21:30:00,2021-07-08 21:39:00,0:09:00
2021-07-08 21:44:00,2021-07-08 22:09:00,0:25:00
2021-07-08 23:05:00,2021-07-08 23:22:00,0:17:00
2021-07-08 23:26:00,2021-07-08 23:44:00,0:18:00
2021-07-09 00:41:00,2021-07-09 01:04:00,0:23:00
2021-07-09 01:08:00,2021-07-09 01:20:00,0:12:00
2021-07-09 02:17:00,2021-07-09 02:46:00,0:29:00
2021-07-09 02:50:00,2021-07-09 02:55:00,0:05:00
2021-07-09 03:52:00,2021-07-09 04:28:00,0:36:00


Other parameters that might be of interest are the `timestamp`, `began` and `completed` values, which allow you to find out when the job was submitted, when it started processing and when it finished. For example:

In [8]:
print(f"Job was submitted at {qj.timestamp}, was queued for {(qj.began - qj.timestamp).seconds}s and took {(qj.completed - qj.began).seconds}s to process.")

Job was submitted at 2021-07-08 19:58:40, was queued for 0s and took 1s to process.


As usual, times reported back are UT. That is it! If all is correct, then the following show the upcoming or current *Swift* visibility window for Sgr A*!

In [9]:
_ = [print(win) for win in qj.result]

2021-07-08 20:03:00 - 2021-07-08 20:33:00 (0:30:00)
2021-07-08 21:30:00 - 2021-07-08 21:39:00 (0:09:00)
2021-07-08 21:44:00 - 2021-07-08 22:09:00 (0:25:00)
2021-07-08 23:05:00 - 2021-07-08 23:22:00 (0:17:00)
2021-07-08 23:26:00 - 2021-07-08 23:44:00 (0:18:00)
2021-07-09 00:41:00 - 2021-07-09 01:04:00 (0:23:00)
2021-07-09 01:08:00 - 2021-07-09 01:20:00 (0:12:00)
2021-07-09 02:17:00 - 2021-07-09 02:46:00 (0:29:00)
2021-07-09 02:50:00 - 2021-07-09 02:55:00 (0:05:00)
2021-07-09 03:52:00 - 2021-07-09 04:28:00 (0:36:00)
2021-07-09 05:28:00 - 2021-07-09 06:07:00 (0:39:00)
2021-07-09 07:03:00 - 2021-07-09 07:42:00 (0:39:00)
2021-07-09 08:39:00 - 2021-07-09 09:18:00 (0:39:00)
2021-07-09 10:15:00 - 2021-07-09 10:54:00 (0:39:00)
2021-07-09 11:50:00 - 2021-07-09 12:29:00 (0:39:00)
2021-07-09 13:26:00 - 2021-07-09 14:05:00 (0:39:00)
2021-07-09 15:02:00 - 2021-07-09 15:40:00 (0:38:00)
2021-07-09 16:37:00 - 2021-07-09 17:16:00 (0:39:00)
2021-07-09 18:14:00 - 2021-07-09 18:52:00 (0:38:00)
2021-07-09 1

That first date is going to be the first visibility date, which if the target is visible now should be essentially the same as `timestamp` in `QueryJob`, or sometime in the future if the target was constrained when the job was submitted. 

In [10]:
print(qj.result[0][0] - qj.timestamp)

0:04:20


Note due to the weird way datetime displays negative timediffs, you might see something like `-1 day, 23:59:19` if the value is negative!